<a href="https://colab.research.google.com/github/st-tran/CSC413-Project/blob/main/CSC413_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC413 Project: GeoGuessr!

## Load in the data

In [ ]:
# 1k Canada dataset
!gdown --id 1QSVxo_GpPQ5gHVh6hkQCEktt6jGX3AoO
!mkdir /content/training
!tar -xzvf 1kimages090180270.tar.gz -C /content/training
!for d in /content/training/images*; do mv "${d}" "/content/training/`basename "${d}" | sed -e 's/images//g' | sed -e 's/\s//g'`"; done
!mv /content/training
!rm -rf /content/1kimages090180270.tar.gz

In [23]:
from typing import *

## Process the data

For each of the images:
- stitch together the 4 pieces ($\{N,E,S,W\}$) into single images
- label the data using latitude/longitude data

In [65]:
from collections import defaultdict
from PIL import Image
from IPython.display import display
from glob import glob
import re
import os

data = defaultdict(list)

def get_image_ids(province: str) -> List[str]:
  images = glob(f"/content/training/{province}/*.jpg")
  ids_no_angle = [f"/content/training/{province}/" + re.sub('_[\\d]*.jpg', '', f.split('/')[-1]) for f in images]
  return ids_no_angle 

def stitch_images_and_save(image_base_path: str) -> None:
  img = Image.new("RGB", (512, 512))
  img_0 = Image.open(f"{image_base_path}_0.jpg")
  img_90 = Image.open(f"{image_base_path}_90.jpg")
  img_180 = Image.open(f"{image_base_path}_180.jpg")
  img_270 = Image.open(f"{image_base_path}_270.jpg")
  img.paste(img_0, (0, 0))
  img.paste(img_90, (256, 0))
  img.paste(img_180, (0, 256))
  img.paste(img_270, (256, 256))

  new_name = f"{image_base_path}_stitched.jpg"
  img.save(new_name)
  data[image_base_path.split("/")[-2]].append(new_name)

In [66]:
for province in glob("/content/training/*"):
  for image in get_image_paths(os.path.basename(province)):
    stitch_images_and_save(image)

FileNotFoundError: ignored

In [62]:
data

defaultdict(list,
            {'Alberta': ['/content/training/Alberta/2009_9T_FOnZo7dppO-KGK6NFaA_stitched.jpg']})